In [1]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [ ]:
class KDE:
    
    def __init__(self, data, n=2**14, MIN=None, MAX=None, plot=False, label=[]):
        self.data = data
        self.n = n
        self.MIN = MIN
        self.MAX = MAX
        self.plot = plot
        self.label = label
        
    